In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [5]:
sc =SparkContext.getOrCreate()

In [6]:
sqlContext = SQLContext(sc)

In [7]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('C:/Users/leicheng/Downloads/daily_weather.csv')

In [8]:
df.columns

['number',
 'air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [10]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
number,1095,547.0,316.24357700987383,0,1094
air_pressure_9am,1092,918.8825513138094,3.184161180386833,907.9900000000024,929.3200000000012
air_temp_9am,1090,64.93300141287072,11.175514003175877,36.752000000000685,98.90599999999992
avg_wind_direction_9am,1091,142.2355107005759,69.13785928889189,15.500000000000046,343.4
avg_wind_speed_9am,1092,5.50828424225493,4.5528134655317185,0.69345139999974,23.554978199999763
max_wind_direction_9am,1092,148.95351796516923,67.23801294602953,28.89999999999991,312.19999999999993
max_wind_speed_9am,1091,7.019513529175272,5.598209170780958,1.1855782000000479,29.84077959999996
rain_accumulation_9am,1089,0.20307895225211126,1.5939521253574893,0.0,24.01999999999907
rain_duration_9am,1092,294.1080522756142,1598.0787786601481,0.0,17704.0


In [11]:
df.describe('air_pressure_9am').show()

+-------+-----------------+
|summary| air_pressure_9am|
+-------+-----------------+
|  count|             1092|
|   mean|918.8825513138094|
| stddev|3.184161180386833|
|    min|907.9900000000024|
|    max|929.3200000000012|
+-------+-----------------+



In [13]:
len(df.columns)

11

In [14]:
df.count()

1095

In [15]:
df2 = df.na.drop(subset=['air_pressure_9am'])

In [16]:
df2.count()

1092

In [18]:
df2.stat.corr("rain_accumulation_9am", "rain_duration_9am")

0.7298253479609021

In [25]:
df.describe('avg_wind_speed_9am').show()

+-------+------------------+
|summary|avg_wind_speed_9am|
+-------+------------------+
|  count|              1092|
|   mean|  5.50828424225493|
| stddev|4.5528134655317185|
|    min|  0.69345139999974|
|    max|23.554978199999763|
+-------+------------------+



In [29]:
df3 = df.na.drop(subset=['rain_accumulation_9am'])
df3.count()

1089

In [30]:
df3.stat.corr("relative_humidity_9am", "relative_humidity_3pm")

0.8830403147328106

In [31]:
removeALLDF = df.na.drop()

In [33]:
removeALLDF.describe(["air_temp_9am"]).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1064|
|   mean| 65.02260949558733|
| stddev|11.168033449415704|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [34]:
removeALLDF.count()

1064

In [35]:
from pyspark.sql.functions import avg

In [36]:
imputeDF = df

In [39]:
for x in imputeDF.columns:
    meanvalue = removeALLDF.agg(avg(x)).first()[0]
    print(x,meanvalue)
    imputeDF = imputeDF.na.fill(meanvalue,[x])

number 545.0018796992481
air_pressure_9am 918.9031798641051
air_temp_9am 65.02260949558733
avg_wind_direction_9am 142.30675564934037
avg_wind_speed_9am 5.48579305071369
max_wind_direction_9am 148.48042413321315
max_wind_speed_9am 6.999713658875691
rain_accumulation_9am 0.18202347650615522
rain_duration_9am 266.3936973996037
relative_humidity_9am 34.07743985327709
relative_humidity_3pm 35.14838093290533


In [40]:
df.describe(["air_temp_9am"]).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287072|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [41]:
imputeDF.describe(["air_temp_9am"]).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1095|
|   mean| 64.93341058219818|
| stddev| 11.14994819992023|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [45]:
df4 = df.na.drop(subset=['rain_accumulation_9am'])
df4.count()

1089

In [46]:
df5 = df.na.drop(subset=['air_temp_9am'])
df5.count()

1090

In [63]:
df2[(df2['air_pressure_9am'] >= 911.736) & (df['air_pressure_9am'] <= 914.67)].count()

80

In [64]:
df2[df2['air_temp_9am'] < 42.292].count()

23

In [12]:
from pyspark import SparkContext 

# check that it really works by running a job
# example from http://spark.apache.org/docs/latest/rdd-programming-guide.html#parallelized-collections
data = range(10000) 
distData = sc.parallelize(data)
distData.filter(lambda x: not x&1).take(10)
# Out: [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]